In [54]:
import pandas as pd
import seaborn as sns; sns.set()
grants = pd.read_csv('open_phil_grants_db_raw.csv', usecols=['Organization Name', 'Focus Area', 'Amount', 'Date'], index_col='Date', parse_dates=True).dropna()
# Map their terminology to standard EA cause areas
cause_map = {'Global Health & Development':'Global Health', 'Farm Animal Welfare':'Animal Welfare', 'Global Catastrophic Risks':'Longtermism', 
                'Potential Risks from Advanced Artificial Intelligence': 'Longtermism', 'Biosecurity and Pandemic Preparedness':'Longtermism', 'Other areas':'Meta'}
mistake_map = {'Eurogroup For Animals':'Eurogroup for Animals', 'Mercy For Animals':'Mercy for Animals', 'IDInsight':'IDinsight', 'IDinsight ':'IDinsight'}
cause_area = list() #store true labels
for i in range(grants.shape[0]):
    fa = grants.iloc[i, 1] #focus area for this row
    org = grants.iloc[i, 0] #org name
    if fa in cause_map:
        cause_area.append(cause_map[fa])
    else:
        cause_area.append('X') #not an EA cause area
    if org == 'GiveWell':
        cause_area[-1] = 'Meta' #supporting GW belongs in Meta, not Global Health
    if org in mistake_map:
        # print(grants.iloc[i, 0])
        grants.iloc[i, 0] = mistake_map[org] #Swap out any mistakes in the OP database
        # print(grants.iloc[i, 0])

grants['Cause Area'] = cause_area
# Create a df of all 'EA' grants
ea_grants = grants[grants['Cause Area'] != 'X'].drop('Focus Area', axis=1)
ea_grants

,Organization Name,Amount,Cause Area
Date,,,
2021-02-01,Crustacean Compassion,786830.0,Animal Welfare
2021-01-01,Center for Security and Emerging Technology,8000000.0,Longtermism
2021-01-01,UC Berkeley,11355246.0,Longtermism
2021-01-01,Berryville Institute of Machine Learning,150000.0,Longtermism
2020-12-01,University of Glasgow,109552.0,Animal Welfare
...,...,...,...
2012-12-01,GiveDirectly,500000.0,Global Health
2012-12-01,Schistosomiasis Control Initiative,250000.0,Global Health
2012-12-01,Against Malaria Foundation,1250000.0,Global Health


In [55]:
unique_orgs = ea_grants['Organization Name'].unique()
unique_orgs.shape

(255,)

In [56]:
# 546 'EA grants', 265 unique, some of which are e.g. universities rather than charitable orgs
ea_grants['org_cause'] = ea_grants.apply(lambda x: x['Organization Name'] + '|' + x['Cause Area'], axis=1)
unique_grants = pd.Series(ea_grants.org_cause.unique())
unique_grants.shape

(261,)

In [17]:
path = 'C:\\Users\\tmaul\\Documents\\GitHub\\ea_stuff\\IRS_data\\'
ea_grants.to_csv(path+'op_ea_grants.csv')

In [22]:
unique_grants.to_csv(path+'org_map_starter.csv')